In [1]:
import os

os.chdir("/Users/np10002274373/ineuron/Projects/CNN_Classifier")

In [2]:

from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

@dataclass(frozen= True) #this decorator with frozen= True makes this behave as a namedtuple
class PrepareCallBackConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [3]:
from deepclassifier.constants import *
from deepclassifier.utils import read_yaml, create_directories
import tensorflow as tf

2022-10-11 21:31:50.713271: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_callback_config(self) -> PrepareCallBackConfig:
        config = self.config.prepare_callback
        model_chkpoint_dir= os.path.dirname(config.checkpoint_model_filepath)

        create_directories([Path(model_chkpoint_dir), Path(config.tensorboard_root_log_dir)])

        prepare_callback_config = PrepareCallBackConfig(
            root_dir= Path(config.root_dir),
            tensorboard_root_log_dir= Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath= Path(config.checkpoint_model_filepath)
            
        )

        return prepare_callback_config

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "PetImages")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [5]:
import time

class PrepareCallBack:
    def __init__(self, config: PrepareCallBackConfig):
        self.config = config

    @property
    def _create_tb_callback(self):
        timestamp= time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir= os.path.join(self.config.tensorboard_root_log_dir, f"tb_logs_at_{timestamp}")
        return tf.keras.callbacks.TensorBoard(log_dir= tb_running_log_dir)

    @property 
    def _create_ckpt_callback(self):
        return tf.keras.callbacks.ModelCheckpoint(filepath=self.config.checkpoint_model_filepath,
        save_best_only= True)

    def get_tb_ckpt_callback(self):
        return [
            self._create_tb_callback,
            self._create_ckpt_callback



        ]
        

In [6]:

import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,           #rescale the pixels
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],     #will not take channel into picture. Only rows and columns
            batch_size=self.config.params_batch_size,           
            interpolation="bilinear"                    #say a 2x2 needs converted to 4x4. Uses interpolation for this
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs                   #data flow's keyword arguments passed
        )

        if self.config.params_is_augmentation:      #In every epoch, the model will see different random version of data as per paramters below
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,           #flip image
                width_shift_range=0.2,
                height_shift_range=0.2,         #stretch the image height wise
                shear_range=0.2,
                zoom_range=0.2,                 #zoom further by 0.2 times
                **datagenerator_kwargs          #apply re-scaling and validation split
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(     #where is the directory
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


    def train(self, callback_list: list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,           #training data
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [7]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallBack(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callback()
    
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator() #training and validation generators being created
    training.train(
        callback_list=callback_list
    )
    
except Exception as e:
    raise e

[2022-10-11 21:31:57,584: INFO: common]: yaml file: configs/config.yaml loaded successfully
[2022-10-11 21:31:57,589: INFO: common]: yaml file: params.yaml loaded successfully
[2022-10-11 21:31:57,592: INFO: common]: created directory at: artifacts
[2022-10-11 21:31:57,593: INFO: common]: created directory at: artifacts/prepare_callback/checkpoint_dir
[2022-10-11 21:31:57,595: INFO: common]: created directory at: artifacts/prepare_callback/tensorboard_root_log_dir
[2022-10-11 21:31:57,597: INFO: common]: created directory at: artifacts/training


2022-10-11 21:31:57.645645: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Found 4998 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.
 536/1250 [===========>..................] - ETA: 29:00 - loss: 9.2029 - accuracy: 0.6210

/Users/np10002274373/ineuron/Projects/CNN_Classifier/env/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:845: UserWarning: Truncated File Read
  warnings.warn(str(msg))


1250/1250 [==============================] - 3892s 3s/step - loss: 6.3601 - accuracy: 0.6919 - val_loss: 6.6717 - val_accuracy: 0.6729


In [8]:
!pip install scipy